假设输入形状为$n_{h}×n_{w}$，卷积核形状为$k_{h}×K_{w}$，那么输出形状将是$(n_{h} - k_{h} + 1)×(n_{w} - k_{w} + 1)$。因此，卷积的输出形状取决于输入形状和卷积核的形状。

填充(padding) 和 步幅(stride) 也会影响输出的大小

padding:

$(n_{h} - k_{h} + p_{h} + 1 ) × (n_{w} - k_{w} + p_{w} + 1)$

原图形为$n_h × n_w$ 卷积核为 $k_h × k_w$  填充大小为 $p_h × p_w$ 
stride:

原图形为$n_h × n_w$ 卷积核为 $k_h × k_w$  填充大小为 $p_h × p_w$ 步幅为$s_h s_w$

$[(n_h - k_h + p_h + s_h) / s_h] × [(n_w - k_w + p_w + s_w) / s_w ]$

当然这个公式不一定准确，当输入可以被步幅整除时，大小为$(n_h / s_h) × (n_w / s_w)$

### 6.3.1 填充
在应用多层卷积时，我们常常丢失边缘像素。多层连续卷积之后累积丢失的像素数量就比较多，解决的办法就是在图像的边界进行元素的填充
添加$p_{h}、p_{w}$行进行填充，输出的形状将为

$(n_{h} - k_{h} + p_{h} + 1 ) × (n_{w} - k_{w} + p_{w} + 1)$

通常，为了使输入输出具有相同的形状，通常设置$p_{h} = k_{h} -1 和 p_{w} = k_{w} - 1$.如果卷积核$k_{h}$是奇数，可以在高度两侧填充相同的的行数。 一般上面添加的比下面多
![填充](https://cdn.staticaly.com/gh/SisyphusTang/Picture-bed@master/20230810/填充.28b7qwxo8y80.webp)


In [2]:
import torch
from torch import nn


# 为了方便起见，我们定义了一个计算卷积层的函数。
# 此函数初始化卷积层权重，并对输入和输出提高和缩减相应的维数
def comp_conv2d(conv2d, X):
    # 这里的（1，1）表示批量大小和通道数都是1
    X = X.reshape((1, 1) + X.shape)  # 形状变为 1，1，X.shape
    Y = conv2d(X)
    # 省略前两个维度：批量大小和通道
    return Y.reshape(Y.shape[2:])

# 请注意，这里每边都填充了1行或1列，因此总共添加了2行或2列
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

### 6.3.2 步幅
有时候为了高效计算或是缩减采样次数，卷积窗口可以跳过中间位置，每次滑动多个元素
当垂直步幅为$s_{h}$，水平步幅为$s_{W}$时，输出形状为

$[(n_h - k_h + p_h + s_h) / s_h] × [(n_w - k_w + p_w + s_w) / s_w ]$

当然这个公式不一定准确，当输入可以被步幅整除时，大小为$(n_h / s_h) × (n_w / s_w)$

In [3]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)

comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

In [4]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))

comp_conv2d(conv2d, X).shape

torch.Size([2, 2])